In [1]:
import pandas as pd
import os

# Adjust this path based on the "Copy Path" option in your Kaggle data sidebar
# Usually it looks like: /kaggle/input/bengaluru-sewage-treatment-and-environmental-data/Bangalore.xlsx
path = "/kaggle/input/bengaluru-sewage-treatment-and-environmental-data/Bangalore.xlsx"

# 1. Check all sheets
xl = pd.ExcelFile(path)
print(f"Available Sheets: {xl.sheet_names}")

# 2. Explore Sheet1 (The main data sheet)
df = pd.read_excel(path, sheet_name='Sheet1')

print("\n--- DATASET SHAPE ---")
print(df.shape)

print("\n--- ALL COLUMN NAMES ---")
# Printing all columns to see the exact naming convention for each STP
for i, col in enumerate(df.columns):
    print(f"{i}: {col}")

print("\n--- FIRST 5 ROWS ---")
display(df.head())

print("\n--- MISSING VALUES SUMMARY ---")
print(df.isnull().sum().sum(), "total missing values")

Available Sheets: ['Sheet1', 'Sheet2', 'Sheet3']

--- DATASET SHAPE ---
(1382, 82)

--- ALL COLUMN NAMES ---
0: S.NO
1: State
2: City/Town
3: Sewage Generation (Million Liters per Day)
4: Installed Capacity (Million Liters per Day)
5: Operational Capacity
6: Temperature (Avg)
7: Max Temperature
8: Min Temperature
9: Date
10: Nagasandra STP_avg_COD
11: Nagasandra  STP_avg_BOD
12: Nagasandra  STP_avg_pH
13: Nagasandra  STP_avg_TSS
14: Nagasandra  STP_avg_Ammonical_Nitrogen
15: Nagasandra  STP_ avg_Total_Nitrogen
16: Nagasandra STP_Installed_Capacity_MLD
17: Nagasandra  STP_Treatment Facility
18: Madiwala  STP_avg_COD
19: Madiwala  STP_avg_BOD
20: Madiwala  STP_avg_pH
21: Madiwala  STP_avg_TSS
22: Madiwala  STP_avg_Ammonical_Nitrogen
23: Madiwala  STP_avg_Total_Nitrogen
24: Madiwala STP_Installed_Capacity_MLD
25: Madiwala  STP_Treatment Facility
26: Hebbal STP_avg_COD
27: Hebbal STP_avg_BOD
28: Hebbal STP_avg_Ph
29: Hebbal STP_avg_TSS
30: Hebbal STP_avg_Ammonical_Nitrogen
31: Hebbal STP_a

,S.NO,State,City/Town,Sewage Generation (Million Liters per Day),Installed Capacity (Million Liters per Day),Operational Capacity,Temperature (Avg),Max Temperature,Min Temperature,Date,...,Cubbon Park STP_Installed_Capacity_MLD,Cubbon Park STP_Treatment Facility,Lalbagh STP_avg_COD,Lalbagh STP_avg_BOD,Lalbagh STP_avg_pH,Lalbagh STP_avg_TSS,Lalbagh STP_avg_Ammonical_Nitrogen,Lalbagh STP_avg_Total_Nitrogen,Lalbagh STP_Installed_Capacity_MLD,Lalbagh STP_Treatment Facility
0,1,Karnataka,Bangaluru,"1,440 MLD",721 MLD,600 MLD,30.630,44.60,16.66,2019-11-01,...,1.5,Membrane,37.994428,4.423286,6.669024,25.082143,1.794402,4.654837,1.5,Extended Aeration + Plate Settlers + UV disinf...
1,2,Karnataka,Bangaluru,"1,440 MLD",721 MLD,600 MLD,30.460,37.35,23.57,2019-11-02,...,1.5,Membrane,62.510314,3.568678,7.822050,22.068409,0.622446,7.031646,1.5,Extended Aeration + Plate Settlers + UV disinf...
2,3,Karnataka,Bangaluru,"1,440 MLD",721 MLD,600 MLD,29.595,34.73,24.46,2019-11-03,...,1.5,Membrane,48.063090,7.673785,6.817205,10.378442,2.414726,6.558920,1.5,Extended Aeration + Plate Settlers + UV disinf...
3,4,Karnataka,Bangaluru,"1,440 MLD",721 MLD,600 MLD,30.965,43.50,18.43,2019-11-04,...,1.5,Membrane,58.038786,8.630831,6.632415,19.037379,2.249366,4.003494,1.5,Extended Aeration + Plate Settlers + UV disinf...
4,5,Karnataka,Bangaluru,"1,440 MLD",721 MLD,600 MLD,29.245,41.33,17.16,2019-11-05,...,1.5,Membrane,52.757032,6.187287,7.715655,29.392509,3.498994,1.483112,1.5,Extended Aeration + Plate Settlers + UV disinf...



--- MISSING VALUES SUMMARY ---
0 total missing values


In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
import re

# 1. LOAD DATA
path = "/kaggle/input/bengaluru-sewage-treatment-and-environmental-data/Bangalore.xlsx"
df = pd.read_excel(path, sheet_name='Sheet1')

# 2. THE CONSOLIDATION LOGIC (Wide to Long)
# List of STP prefixes based on your column exploration
stps = [
    "Nagasandra", "Madiwala", "Hebbal", "Yelahanka", 
    "Jakkur", "Rajacanal", "K.R. Puram", "Cubbon Park", "Lalbagh"
]

all_data = []

for stp in stps:
    # We use Regex to find columns because of the double spaces and 'Ph'/'pH' typos
    # Pattern explanation: Start with STP name, followed by any space, then 'STP_avg_' then the metric
    def get_col(metric_pattern):
        pattern = re.compile(rf"{re.escape(stp)}.*STP_.*{metric_pattern}", re.IGNORECASE)
        match = [c for c in df.columns if pattern.search(c)]
        return match[0] if match else None

    cols = {
        'pH': get_col('pH'),
        'COD': get_col('COD'),
        'BOD': get_col('BOD'),
        'TSS': get_col('TSS'),
        'Ammonia': get_col('Ammonical_Nitrogen')
    }

    # Only process if essential columns are found
    if cols['pH'] and cols['COD']:
        subset = df[['Date', 'Temperature (Avg)', 'Max Temperature', 'Min Temperature']].copy()
        subset['pH'] = df[cols['pH']]
        subset['COD'] = df[cols['COD']]
        subset['BOD'] = df[cols['BOD']]
        subset['TSS'] = df[cols['TSS']]
        subset['Ammonia'] = df[cols['Ammonia']]
        subset['Location'] = stp
        all_data.append(subset)

# Combine all 9 STPs into one master training set
train_df = pd.concat(all_data, ignore_index=True)

# 3. CLEANING & FEATURE ENGINEERING
# Convert Date
train_df['Date'] = pd.to_datetime(train_df['Date'])
train_df['Month'] = train_df['Date'].dt.month

# Feature 1: Temperature Delta (Impacts biological activity)
train_df['Temp_Range'] = train_df['Max Temperature'] - train_df['Min Temperature']

# Drop rows with NaNs if any were introduced during the regex match
train_df.dropna(subset=['pH', 'TSS', 'COD', 'BOD'], inplace=True)

# 4. MODEL TRAINING (Multi-Output Regression)
# Inputs: pH, TSS (Turbidity), Avg Temp, Month
X = train_df[['pH', 'TSS', 'Temperature (Avg)', 'Month', 'Temp_Range']]
# Targets: COD, BOD, Ammonia
y = train_df[['COD', 'BOD', 'Ammonia']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Using XGBoost for high accuracy on tabular sewage data
model = MultiOutputRegressor(xgb.XGBRegressor(
    n_estimators=1000,
    max_depth=7,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=-1,
    tree_method='hist' # Faster on Kaggle
))

print(f"🚀 Training HarvesSink Brain on {len(train_df)} data points from 9 Bangalore STPs...")
model.fit(X_train, y_train)

# 5. EVALUATION
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
r2 = r2_score(y_test, y_pred, multioutput='raw_values')

print("\n--- MODEL ACCURACY REPORT ---")
metrics = ['COD', 'BOD', 'Ammonia']
for i, m in enumerate(metrics):
    print(f"{m} -> MAE: {mae[i]:.2f} mg/L | R² Score: {r2[i]:.4f}")

# 6. EXPORT
joblib.dump(model, 'harvessink_v1_bangalore.joblib')
print("\n✅ harvessink_v1_bangalore.joblib exported successfully.")

🚀 Training HarvesSink Brain on 12438 data points from 9 Bangalore STPs...

--- MODEL ACCURACY REPORT ---
COD -> MAE: 7.77 mg/L | R² Score: -0.0785
BOD -> MAE: 2.17 mg/L | R² Score: -0.1168
Ammonia -> MAE: 1.18 mg/L | R² Score: -0.1044

✅ harvessink_v1_bangalore.joblib exported successfully.


In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
import re

# 1. LOAD & TIDY (Same as before but keeping Location)
path = "/kaggle/input/bengaluru-sewage-treatment-and-environmental-data/Bangalore.xlsx"
df = pd.read_excel(path, sheet_name='Sheet1')

stps = ["Nagasandra", "Madiwala", "Hebbal", "Yelahanka", "Jakkur", "Rajacanal", "K.R. Puram", "Cubbon Park", "Lalbagh"]
all_data = []

for stp in stps:
    def get_col(metric_pattern):
        pattern = re.compile(rf"{re.escape(stp)}.*STP_.*{metric_pattern}", re.IGNORECASE)
        match = [c for c in df.columns if pattern.search(c)]
        return match[0] if match else None

    cols = {'pH': get_col('pH'), 'COD': get_col('COD'), 'BOD': get_col('BOD'), 'TSS': get_col('TSS'), 'Ammonia': get_col('Ammonical_Nitrogen')}
    
    if cols['pH'] and cols['COD']:
        subset = df[['Temperature (Avg)', 'Max Temperature', 'Min Temperature']].copy()
        subset['pH'], subset['COD'], subset['BOD'], subset['TSS'], subset['Ammonia'] = df[cols['pH']], df[cols['COD']], df[cols['BOD']], df[cols['TSS']], df[cols['Ammonia']]
        subset['STP_Location'] = stp # Keep the name!
        all_data.append(subset)

train_df = pd.concat(all_data, ignore_index=True).dropna()

# 2. THE FIX: ONE-HOT ENCODING
# This tells the model "This data is from Hebbal" vs "This is from Jakkur"
train_df = pd.get_dummies(train_df, columns=['STP_Location'])

# 3. PREPARE X and Y
# We exclude the raw COD/BOD/Ammonia from X
X = train_df.drop(columns=['COD', 'BOD', 'Ammonia'])
y = train_df[['COD', 'BOD', 'Ammonia']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. TUNED MODEL
model = MultiOutputRegressor(xgb.XGBRegressor(
    n_estimators=1500,
    max_depth=9,
    learning_rate=0.01, # Slower learning to find the pattern
    subsample=0.9,
    colsample_bytree=0.9,
    objective='reg:squarederror'
))

print("🧠 Training V2: Location-Aware HarvesSink Brain...")
model.fit(X_train, y_train)

# 5. EVALUATION
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
r2 = r2_score(y_test, y_pred, multioutput='raw_values')

print("\n--- V2 ACCURACY REPORT ---")
for i, m in enumerate(['COD', 'BOD', 'Ammonia']):
    print(f"{m} -> MAE: {mae[i]:.2f} | R²: {r2[i]:.4f}")

joblib.dump(model, 'harvessink_v2.joblib')
# Save the column names so your laptop app knows how to format the input!
joblib.dump(X.columns.tolist(), 'model_features.joblib')

🧠 Training V2: Location-Aware HarvesSink Brain...

--- V2 ACCURACY REPORT ---
COD -> MAE: 6.82 | R²: 0.1918
BOD -> MAE: 2.05 | R²: 0.0737
Ammonia -> MAE: 1.13 | R²: -0.0165


['model_features.joblib']